In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
datapath = '/content/drive/MyDrive/Omdena-India-Socio-Economy/'
#file1 = datapath + 'tweet_with_transformer_labels.csv'
file1 = datapath + 'Tweet_With_TextBlob_labels.csv'

In [4]:
import pandas as pd
import numpy as np
from collections import Counter

In [5]:
data = pd.read_csv(file1)

In [6]:
len(data['text'])

109533

In [7]:
a = list(data['text'])
label = np.array((data['label']))

In [8]:
# Convert list to collection of strings
alltext = ' '.join([c for c in a]) 
# Convert 
words = alltext.split(' ')

In [9]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

In [10]:
text_to_int = []
for i, tweet in enumerate(a):
  text_to_int.append([vocab_to_int[word] for word in tweet.split(' ')])

In [11]:
a[10]

'support fight corona viruse jai hind'

In [12]:
text_to_int[10]

[110, 36, 2, 4803, 284, 450]

In [13]:
vocab_to_int['fight']

36

In [14]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  
print()

# print tokens in first review
print('Tokenized tweet: \n', text_to_int[:10])

Unique words:  60398

Tokenized tweet: 
 [[4900, 291, 441, 2, 48, 10105, 123], [1107, 744, 18, 685, 2837, 942, 7478, 30245], [2594, 3896, 31, 79, 1242, 60, 914, 847, 61, 558, 43, 67, 133, 3897, 1827, 22546, 578, 1767, 848, 59, 1419, 600, 1351, 470, 762, 2, 75], [9, 603, 3820, 13, 688], [111, 544, 161, 56, 70, 1780, 31, 133, 13288, 8364, 100, 2119, 1529, 2119, 100, 839, 754, 87, 5, 2, 105, 1562, 189, 252, 284, 450], [16264, 41, 5972, 60, 5097, 1518, 43, 489, 349, 228, 4187, 403, 133, 407, 3897, 578, 1767, 848, 59, 13, 156, 600, 1266, 470, 2031, 2, 75, 3771, 1039, 165, 96], [2594, 3896, 31, 79, 1242, 60, 914, 847, 61, 558, 43, 67, 133, 3897, 1827, 22546, 578, 1767, 848, 59, 1419, 600, 1351, 470, 762, 2, 75], [41, 5548, 22, 60, 1735, 1408, 133, 225, 4, 282, 403, 133, 1827, 407, 225, 578, 1767, 848, 59, 13, 156, 332, 22, 257, 341, 470, 393, 2, 75], [16264, 22, 60, 1735, 1408, 133, 225, 4, 282, 403, 133, 1827, 407, 225, 578, 1767, 848, 59, 13, 156, 332, 22, 257, 341, 470, 393, 2, 75], [36, 

In [15]:
# Check that there are no zero length tweets. - these should have been removed during labeling but good to check
zero_idx = [ii for ii, tweet in enumerate(text_to_int) if len(tweet) == 0]
len(zero_idx)

0

Padding **sequences**

In [16]:
# Count the length of tweets
tweet_lens = Counter([len(x) for i, x in enumerate(text_to_int)])
print("Maximum review length: {}".format(max(tweet_lens)))

Maximum review length: 51


In [17]:
seq_length = max(tweet_lens)

def pad_features(text_to_int, seq_length):

  features = np.zeros((len(text_to_int), seq_length), dtype=int)
  for i, row in enumerate(text_to_int):
    features[i, -len(row):] = np.array(row)[:seq_length]

  return features

In [18]:
features = pad_features(text_to_int, seq_length)

In [19]:
## test statements - do not change - ##
assert len(features)==len(text_to_int), "Features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

## **Training**, **Validation**, **Test**

In [20]:
split_frac = 0.8
split_idx = int(len(features)*split_frac)
split_idx

87626

In [21]:
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = label[:split_idx], label[split_idx:]

In [22]:
test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

In [23]:
## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(87626, 51) 
Validation set: 	(10953, 51) 
Test set: 		(10954, 51)


**DataLoaders and Batching**



In [24]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [25]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [26]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))


# dataloaders
batch_size = 128

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [27]:
len(train_x)

87626

In [28]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([128, 51])
Sample input: 
 tensor([[    0,     0,     0,  ...,  1638,   575,   579],
        [    0,     0,     0,  ...,  1115, 47277,   382],
        [    0,     0,     0,  ...,   595,  5126,     6],
        ...,
        [    0,     0,     0,  ...,    61,  2897,    73],
        [    0,     0,     0,  ...,   152,   934,   809],
        [    0,     0,     0,  ...,   352,     2,    10]])

Sample label size:  torch.Size([128])
Sample label: 
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 1, 1, 0])


# Sentiment Network with PyTorch

In [29]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [30]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2
epochs = 4

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(60399, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


## Training

In [31]:
# loss and optimization functions
lr=0.01

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [32]:
counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.346826... Val Loss: 0.393772
Epoch: 1/4... Step: 200... Loss: 0.361022... Val Loss: 0.299911
Epoch: 1/4... Step: 300... Loss: 0.170600... Val Loss: 0.285974
Epoch: 1/4... Step: 400... Loss: 0.229966... Val Loss: 0.226250
Epoch: 1/4... Step: 500... Loss: 0.263847... Val Loss: 0.195669
Epoch: 1/4... Step: 600... Loss: 0.282685... Val Loss: 0.203124
Epoch: 2/4... Step: 700... Loss: 0.074071... Val Loss: 0.178758
Epoch: 2/4... Step: 800... Loss: 0.099148... Val Loss: 0.185602
Epoch: 2/4... Step: 900... Loss: 0.071918... Val Loss: 0.159428
Epoch: 2/4... Step: 1000... Loss: 0.072795... Val Loss: 0.161333
Epoch: 2/4... Step: 1100... Loss: 0.072440... Val Loss: 0.146273
Epoch: 2/4... Step: 1200... Loss: 0.075999... Val Loss: 0.148477
Epoch: 2/4... Step: 1300... Loss: 0.094611... Val Loss: 0.141851
Epoch: 3/4... Step: 1400... Loss: 0.089874... Val Loss: 0.150281
Epoch: 3/4... Step: 1500... Loss: 0.130800... Val Loss: 0.158352
Epoch: 3/4... Step: 1600... Loss: 

## Testing

In [33]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.145
Test accuracy: 0.944
